In [2]:
import numpy as np
import pandas as pd


In [3]:
# Load the dataset
file_path = 'datasets/claude_5k.csv'
data = pd.read_csv(file_path).values

In [4]:
print(data[1,50])

4.0


In [5]:
##visulize data
features = data[1,:]
print(features)

[  1.       183.19193    4.        25.         0.         1.
 193.33936   12.        22.37266    0.         1.       205.07791
  12.        23.945366   0.         1.       214.5979     4.
  22.783577   0.         1.       225.87688    4.        21.84085
   0.         1.       235.90842    0.        21.850979   0.
   1.       246.12164   12.        23.093002   0.         1.
 257.69952   12.        22.847837   0.         1.       268.07233
   8.        21.526155   0.         1.       278.74005   12.
  21.072702   0.         4.      ]


ego features extract


In [6]:
row = data[1,:]
# Ego vehicle features
ego_features = row[:5]
ego_lane = ego_features[2]//4 +1  # Lane ID of ego vehicle
ego_speed = ego_features[3]  # Speed of ego vehicle
print(row)

[  1.       183.19193    4.        25.         0.         1.
 193.33936   12.        22.37266    0.         1.       205.07791
  12.        23.945366   0.         1.       214.5979     4.
  22.783577   0.         1.       225.87688    4.        21.84085
   0.         1.       235.90842    0.        21.850979   0.
   1.       246.12164   12.        23.093002   0.         1.
 257.69952   12.        22.847837   0.         1.       268.07233
   8.        21.526155   0.         1.       278.74005   12.
  21.072702   0.         4.      ]


extract other vehicles

In [13]:
row = data[1,:]
other_vehicles = row[5:50].reshape(9, 5)  # 9 vehicles, 5 features each

# Separate features of other vehicles
other_lanes = other_vehicles[:, 2] // 4 + 1  # Lane IDs of other vehicles
distances = np.abs(other_vehicles[:, 1] - ego_features[1])  # Distances from ego vehicle
relative_velocities = other_vehicles[:, 3] - ego_speed  # Relative velocities
print(other_vehicles)

[[  1.       193.33936   12.        22.37266    0.      ]
 [  1.       205.07791   12.        23.945366   0.      ]
 [  1.       214.5979     4.        22.783577   0.      ]
 [  1.       225.87688    4.        21.84085    0.      ]
 [  1.       235.90842    0.        21.850979   0.      ]
 [  1.       246.12164   12.        23.093002   0.      ]
 [  1.       257.69952   12.        22.847837   0.      ]
 [  1.       268.07233    8.        21.526155   0.      ]
 [  1.       278.74005   12.        21.072702   0.      ]]


calculations

In [8]:
# Number of vehicles in ego lane and adjacent lanes
vehicles_in_ego_lane = np.sum(other_lanes == ego_lane)
vehicles_in_left_lane = np.sum(other_lanes == ego_lane - 1)
vehicles_in_right_lane = np.sum(other_lanes == ego_lane + 1)

print(vehicles_in_ego_lane)

2


In [9]:
# Closest vehicles
closest_ego_index = np.where(other_lanes == ego_lane, distances, np.inf).argmin() if vehicles_in_ego_lane !=0 else np.NAN
closest_left_index = np.where(other_lanes == ego_lane - 1, distances, np.inf).argmin() if vehicles_in_left_lane !=0 else np.NAN
closest_right_index = np.where(other_lanes == ego_lane + 1, distances, np.inf).argmin() if vehicles_in_right_lane !=0 else np.NAN
print(closest_ego_index)


2


In [10]:
if np.isnan(closest_ego_index):
    closest_in_ego_lane_dist = np.inf  # Assign default value for no vehicle
else:
    closest_in_ego_lane_dist = distances[int(closest_ego_index)]  # Use int() to ensure valid index
print(closest_in_ego_lane_dist)

31.405969999999996


In [11]:
if np.isnan(closest_ego_index):
    relative_velocity_ego_lane = 10000  # Assign large value for no vehicle
else:
    relative_velocity_ego_lane = relative_velocities[int(closest_ego_index)]  
print(relative_velocity_ego_lane)

-2.216422999999999


In [12]:
relative_velocity_closest_ego = relative_velocities[closest_ego_index] if closest_in_ego_lane < len(relative_velocities) else 0
relative_velocity_closest_left = relative_velocities[closest_left_index] if closest_left_lane < len(relative_velocities) else 0
relative_velocity_closest_right = relative_velocities[closest_right_index] if closest_right_lane < len(relative_velocities) else 0
print(relative_velocity_closest_ego)

NameError: name 'closest_in_ego_lane' is not defined

no vehivle handeling


In [ ]:
if np.any(np.where(other_lanes == ego_lane, distances, np.inf) != np.inf):
    print("There are valid vehicles in the lane ego_lane + 3.")
else:
    print("No valid vehicles in the lane ego_lane + 3.")

There are valid vehicles in the lane ego_lane + 3.
